In [1]:
import pandas as pd
import numpy as np
from aux_fun import *

In [2]:
# Display only 2 decimals on pandas data frames
pd.options.display.float_format = '{:.2f}'.format

# Display all cell results (not only last one)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Lendo dados de treino
dados = pd.read_csv('https://raw.githubusercontent.com/Cayan-Portela/ceub/main/dados/bank_customer_treino.csv')
dados.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15765192,564,France,Male,26,7,84006.88,2,0,0,183490.99,0
1,15631882,688,Germany,Male,45,9,103399.87,1,0,0,129870.93,0
2,15777586,784,Spain,Female,42,2,109052.04,2,1,0,6409.55,0
3,15577107,657,Spain,Female,22,6,0.00,3,0,1,168412.07,1
4,15722731,653,France,Male,46,0,119556.10,1,1,0,78250.13,1


In [4]:
# pre-process da coluna gender
dados['gender'] = np.where(dados.gender == "Male", 1,  0)

In [5]:
# covariaveis (col_x) e variavel target (col_y)
col_x = ['credit_score', 'gender', 'age', 'credit_card']
col_y = 'churn'

In [6]:
y = dados[col_y]
X_mat = matriz_x(colunas=col_x, dados=dados)

Newton-Raphson

$\beta^{(k+1)} = $

In [7]:
# Iteracoes 
betas = beta_inicial(X_mat)
p = calcula_p(X=X_mat, B=betas)
W = np.diag(p)


for i in range(20):
    p =  calcula_p(X=X_mat, B=betas)
    W = np.diag(p)
    beta_k1 = beta_update(X=X_mat, W=W, y=y, p=p)
    betas = betas + beta_k1

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
sk_logistica = LogisticRegression()
sk_logistica.fit(dados[col_x], y)

LogisticRegression()

In [10]:
print(f"Nossos Betas: {betas}")
print(f"Sklearn Betas: {sk_logistica.intercept_, sk_logistica.coef_}")

Nossos Betas: [-3.25658521e+00 -6.58125817e-04 -4.89180017e-01  6.40564764e-02
 -4.90816166e-02]
Sklearn Betas: (array([-3.27061466]), array([[-0.00063749, -0.49520859,  0.0641967 , -0.052929  ]]))


Achar a probabilidade de churn de cada cliente na base de teste

In [11]:
dados_teste = pd.read_csv('https://raw.githubusercontent.com/Cayan-Portela/ceub/main/dados/bank_custoter_teste.csv')
dados_teste['gender'] = np.where(dados_teste.gender == "Male", 1, 0)

In [12]:
X_mat_teste = matriz_x(colunas=col_x, dados=dados_teste)
p_teste = calcula_p(X=X_mat_teste, B=betas)

In [14]:
# ordernando as probabilidades "p_teste" da maior para a menor
# olhando as 10 maiores probabilidades estimadas
np.sort(p_teste)[::-1][:10]

array([0.86478835, 0.83914253, 0.83676744, 0.83495899, 0.82278887,
       0.80055285, 0.79392211, 0.78976356, 0.78309564, 0.77431609])

In [24]:
pd.crosstab(
    np.where(p_teste >= 0.5, 1, 0),
    dados_teste['churn']
)

churn,0,1
row_0,,
0,3089,753
1,116,42


In [28]:
dados_teste.churn.value_counts(normalize=True)

0   0.80
1   0.20
Name: churn, dtype: float64

In [29]:
np.sqrt(10)

3.1622776601683795

In [54]:
def x_update(x):
    #return (1/2) * (x  + 10/x)
    return ((x**2)-10) / (2*x)

In [55]:
x_ = 3

for i in range(20):
    x_ = x_ - x_update(x_)

In [56]:
x_
np.sqrt(10)

3.1622776601683795

3.1622776601683795